## SWARMAURI RETRIEVAL AUGMENTED GENERATION AGENT (RAG)

In [2]:
# Import GroqMOdel using api key from groq and initializing.

In [3]:
from swarmauri.llms.concrete.GroqModel import GroqModel
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("GROQ_API_KEY") 
if API_KEY:
    llm = GroqModel(api_key=API_KEY)
    print("LLM Initialized Successfully")
else:
    print("API Key not found!")

LLM Initialized Successfully


Setting up vector store

In [5]:
# from swarmauri.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore
# from swarmauri.documents.concrete.Document import Document

# vector_store= TfidfVectorStore()

# documents = [Document(content="Their sister's name is Jane."),
#     Document(content="Their mother's name is Jean."),
#     Document(content="They use AWS for cloud services."),
#     Document(content="Their grandather's name is Alex.")]

# # Adding documents to the store
# vector_store.add_documents(documents)

# print("Documents successfully added to the vector store!")

# # Retrieve and print all documents
# all_docs = vector_store.get_all_documents()
# print("All documents in the victor store:")
# for doc in all_docs:
#     print(doc.content)

# print(f"{len(vector_store.documents)} documents added to the vector store.")

Loading pdfs document

In [7]:
from swarmauri.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore
from swarmauri.documents.concrete.Document import Document

vector_store= TfidfVectorStore()

documents = [Document(content="Their sister's name is Jane."),
    Document(content="Their mother's name is Jean."),
    Document(content="They use AWS for cloud services."),
    Document(content="Their grandather's name is Alex.")]

# Adding documents to the store
vector_store.add_documents(documents)

print("Documents successfully added to the vector store!")

# Retrieve and print all documents
all_docs = vector_store.get_all_documents()
print("All documents in the victor store:")
for doc in all_docs:
    print(doc.content)

print(f"{len(vector_store.documents)} documents added to the vector store.")

Documents successfully added to the vector store!
All documents in the victor store:
Their sister's name is Jane.
Their mother's name is Jean.
They use AWS for cloud services.
Their grandather's name is Alex.
4 documents added to the vector store.


# Building the RAG Agent

First step is setting up vector store to hold external knowledge in document form. RAG agent will use store to retrieve relevant info during query processing.

In [10]:
from swarmauri.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.messages.concrete.SystemMessage import SystemMessage
from swarmauri.messages.concrete.HumanMessage import HumanMessage

In [11]:
# Create a system mesage
system_context = SystemMessage(content="Your name is Jeff.")

In [12]:
# Initialize the conversation
conversation = MaxSystemContextConversation(system_context=system_context, max_size=4)

In [13]:
# Add a user message
user_message = HumanMessage(content="What is my name?")
conversation.add_message(user_message)

In [14]:
# Print the current converstion context
print("Current conversation history:")
for message in conversation.history:
    print(message.content)

Current conversation history:
Your name is Jeff.
What is my name?


Integrating the Language Model (LLM)

In [16]:
import os
from swarmauri.llms.concrete.GroqModel import GroqModel as LLM
from swarmauri.conversations.concrete.Conversation import Conversation
from dotenv import load_dotenv


load_dotenv()

True

In [17]:
API_KEY = os.getenv("GROQ_API_KEY")
if API_KEY:
    llm = GroqModel(api_key=API_KEY)
    print("LLM Initialized Successfully")
else:
    print("API Key not found!")

LLM Initialized Successfully


Function to get allowed models

In [19]:
# Function to get allowed models, filtering out failing ones
def get_allowed_models(llm):
    failing_llms = [
        "llama3-70b-8192",
        "llama3.2-90b-text-preview",
        "mixtral-8x7b-32768",
        "llava-v1.5-7b-4096-preview",
        "llama-guard-3-8b"
    ]
    return [model for model in llm.allowed_models if model not in failing_llms]

In [20]:
# Initialize the GroqModel
llm = LLM(api_key=API_KEY)

In [21]:
# Print model informaion
print(f"Resource: {llm.resource}")
print(f"Type: {llm.type}")
print(f"Default Name: {llm.name}")

Resource: LLM
Type: GroqModel
Default Name: gemma-7b-it


In [22]:
# Get allowed models
allowed_models = get_allowed_models(llm)
print("Allowed Models:", allowed_models)

Allowed Models: ['gemma-7b-it', 'gemma2-9b-it', 'llama-3.1-70b-versatile', 'llama-3.1-8b-instant', 'llama-3.2-11b-text-preview', 'llama-3.2-1b-preview', 'llama-3.2-3b-preview', 'llama-3.2-90b-text-preview', 'llama3-8b-8192', 'llama3-groq-70b-8192-tool-use-preview', 'llama3-groq-8b-8192-tool-use-preview']


Example usage with no system context

In [24]:
# Set the model name to the first available allowed model
llm.name = allowed_models[0]

In [25]:
# Create conversation
conversation = Conversation()

In [26]:
# Add a human message
input_data = "Hello"
human_message = HumanMessage(content=input_data)
conversation.add_message(human_message)

In [27]:
# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with no system context for {llm.name}: {prediction}")

Prediction with no system context for gemma-7b-it: Hello! 👋 It's great to hear from you. How can I help you today? 😊


Example Usage with a System Context

In [29]:
# Example usage with a system context
system_context = "You are an assistant that provides answers to the user."
conversation = MaxSystemContextConversation(system_context=SystemMessage(content=system_context), max_size=2)

In [30]:
# Create a human message with the content "Hi" and add it to the conversation
human_message = HumanMessage(content="Hi")
conversation.add_message(human_message)

In [31]:
# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with no system context for {llm.name}: {prediction}")

Prediction with no system context for gemma-7b-it: Greetings! I am ready to assist you with any questions or tasks you may have. How can I help you today?


Building the RAG Agent

In [33]:
from swarmauri.agents.concrete.RagAgent import RagAgent

In [34]:
# Created a new system context for the Rag Agent
rag_system_context = "You are an assistant that provides answers to the user. You utilize the details below:"
# Create a new conversation for RAG Agent
rag_conversation = MaxSystemContextConversation(system_context=SystemMessage(content=rag_system_context), max_size=4)

# Initialize the RAG Agent by combining LLM, conversation and vector store
rag_agent = RagAgent(
    llm = llm,
    conversation = rag_conversation,
    system_context=rag_system_context,
    vector_store=vector_store,
)

Handling Queries with the RAG Agent

In [36]:
# Test the agent with different queris
queries = [
    "What cloud service do they use?",
    "What is their mother's name?",
]

for query in queries:
    response = rag_agent.exec(query)
    print(f"Query: {query}\nRAG Agent Response: {response}\n")

Query: What cloud service do they use?
RAG Agent Response: The provided text does not include information regarding the specific cloud service used by the user, so I am unable to answer this question from the given context.

Query: What is their mother's name?
RAG Agent Response: Their mother's name is Jean.



In [37]:
pip freeze

aext-assistant @ file:///private/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_24994515xd/croot/aext-assistant_1717062163042/work
aext-assistant-server @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_4boob09gv8/croot/aext-assistant-server_1717059973985/work
aext-core @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_3etmw5tx6d/croot/aext-core_1717056386167/work
aext-core-server @ file:///private/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_68sv0mesk_/croot/aext-core-server_1717075032630/work
aext-panels @ file:///private/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_892vjkdux1/croot/aext-panels_1717062179502/work
aext-panels-server @ file:///private/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_7cdpxd64im/croot/aext-panels-server_1717060126704/work
aext-share-notebook @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_e5jlgefh0_/croot/aext-share-notebook_1717059291973/work
aext-share-notebook-server @